In [8]:
! pip install pdfplumber


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\jimmy\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [51]:
!pip install pymongo


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\jimmy\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [88]:
import re

from PyPDF2 import PdfReader

from transformers import GPT2TokenizerFast
 
# Load tokenizer to count tokens

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
 
# Load PDF

doc_path = r"C:\Users\jimmy\Downloads\N1_Trunk_Infra_Agreement1.pdf"

reader = PdfReader(doc_path)
 
# Extract all text

full_text = ""

for page in reader.pages:

    full_text += page.extract_text() + "\n"
 
# Define section patterns

patterns = {

    "Agreement Metadata": r"(?is)(agreement.*?value:.*?)(?=Item No.|Scope|Staff|Personnel|Equipment|Notes|Technical)",

    "Scope of Work": r"(?is)(scope of work.*?)(?=Item No.|Staff|Personnel|Equipment|Notes|Technical)",

    "Staffing/Personnel Requirements": r"(?is)(Item No.*?Manager.*?)(?=Technical Requirements|Equipment|Notes|conditions)",

    "Equipment Requirements": r"(?is)(Technical Requirements.*?)(?=Chief Engineer|Notes|conditions)",

    "Notes and Conditions": r"(?is)(Note:.*)"

}
 
# Function to split long sections into chunks

def chunk_section(text, section_name, max_tokens=2000, small_chunk=800, overlap=100, metadata=None):

    tokens = tokenizer.encode(text)

    chunks = []
 
    if len(tokens) <= max_tokens:

        chunks.append({"section": section_name, "metadata": metadata or {}, "content": text.strip()})

    else:

        start = 0

        while start < len(tokens):

            end = min(start + small_chunk, len(tokens))

            chunk_text = tokenizer.decode(tokens[start:end])

            chunks.append({"section": section_name, "metadata": metadata or {}, "content": chunk_text.strip()})

            start += small_chunk - overlap

    return chunks
 
all_chunks = []
 
for name, pattern in patterns.items():
    match = re.search(pattern, full_text, re.DOTALL)
    if match:
        section_text = match.group().strip()

        if name == "Staffing/Personnel Requirements":
            rows = re.findall(r"\d+\.\s*(.*?)\s*-.*", section_text)
            for row in rows:
                metadata = {"role": row.strip()}
                section_chunks = chunk_section(section_text, name, metadata=metadata)
                all_chunks.extend(section_chunks)

        elif name == "Equipment Requirements":
            items = re.findall(r"\d+\.\s*([A-Za-z0-9\- ]+)-", section_text)
            for item in items:
                metadata = {"item": item.strip()}
                section_chunks = chunk_section(section_text, name, metadata=metadata)
                all_chunks.extend(section_chunks)

        else:
            section_chunks = chunk_section(section_text, name)
            all_chunks.extend(section_chunks)
    else:
        all_chunks.append({"section": name, "metadata": {}, "content": "<Not Found>"})
 
# Show enriched chunks

for c in all_chunks[:10]:  # preview first 10

    print("\n=====", c["section"], "=====")

    print("Metadata:", c["metadata"])

    print(c["content"][:30], "...\n")

 

Token indices sequence length is longer than the specified maximum sequence length for this model (1140 > 1024). Running this sequence through the model will result in indexing errors



===== Agreement Metadata =====
Metadata: {}
AGREEMENT
GOVERNMENT OF ANDHRA ...


===== Scope of Work =====
Metadata: {}
scope of works. The Bidder sha ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Project Manager'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Transportation Manager'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Structural Manager'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Manager – QS & Billing'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Manager – QA & QC / Materials'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Construction Manager'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Manager – Construction safet

In [89]:
import json

# Save all_chunks to a JSON file
output_path = r"C:\Users\jimmy\Downloads\N1_Trunk_Infra_Agreement_chunks.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)

print(f"Chunks saved to {output_path}")

Chunks saved to C:\Users\jimmy\Downloads\N1_Trunk_Infra_Agreement_chunks.json


In [90]:
%pip install openai
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [91]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
import tiktoken
import concurrent.futures
import logging

load_dotenv("../injestion/config/.env")
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Helper functions for embeddings and similarity
def get_openai_embedding(text, timeout=15):
    """Get embeddings using Azure OpenAI's text-embedding model with context window truncation and timeout."""
    # Truncate text to fit within model context window (e.g., 8000 tokens for text-embedding-3-small)
    max_tokens = 8000
    encoding = tiktoken.encoding_for_model("text-embedding-3-small")
    tokens = encoding.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        text = encoding.decode(tokens)
    def call():
        return client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        ).data[0].embedding
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(call)
        try:
            return future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            logging.error(f"OpenAI embedding call timed out for text: {text[:50]}")
            raise TimeoutError("OpenAI embedding call timed out.")

In [92]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [93]:
import sys
import os
from datetime import datetime

# Add the injestion directory to Python path
sys.path.append(os.path.abspath("../injestion"))

from models.datamodel_pdantic import Module, KnowledgeObject, Chunk, EmbeddingMeta, Metadata
from scripts.mongo_utils import (
    insert_module, 
    insert_knowledge_object, 
    insert_chunk, 
    create_embedding_meta
)

# 1. Create and insert Module
module_data = {
    "module_id": "N1_Trunk_Infra_Agreement",
    "module_tag": ["infrastructure", "construction", "amaravati", "roads"],
    "module_link": ["https://example.com/n1-trunk-agreement"]
}

try:
    module_obj = Module(**module_data)
    module_id = insert_module(module_obj)  # This returns the MongoDB ObjectId as string
    print(f"✅ Module created with ID: {module_id}")
except Exception as e:
    print(f"❌ Module creation failed: {e}")
    module_id = "N1_Trunk_Infra_Agreement"  # fallback

# 2. Create Metadata for Knowledge Object
metadata = Metadata(
    path="/documents/N1_Trunk_Infra_Agreement.pdf",
    repo_url="https://github.com/your-repo/documents",
    intent_category="construction_agreement",
    version="1.0",
    modified_time=datetime.utcnow(),
    csp="amaravati_development"
)

# 3. Create and insert Knowledge Object
knowledge_object_data = {
    "title": "N1 Trunk Infrastructure Agreement",
    "named_entity": "BSR Infratech India Limited",
    "summary": "Construction agreement for smart trunk infrastructure in Amaravati Capital City including roads, storm water drains, water supply network, sewerage network, utility ducts, and other infrastructure components.",
    "content": "Agreement for construction of balance smart trunk infrastructure with Roads, Storm Water Drains, Water Supply Network, Sewerage Network, Utility Ducts for Power& ICT, Reuse waterline, Pedestrian tracks, Cycle tracks, Avenue Plantation& Street Furniture etc. in N1 Road in Amaravati Capital City",
    "keywords": "infrastructure, construction, amaravati, roads, drainage, utilities, BSR Infratech",
    "texts": " ".join([chunk["content"] for chunk in all_chunks if chunk["content"] != "<Not Found>"]),
    "is_terraform": False,
    "metadata": metadata,
    "module_id": module_id,  # Use the actual ObjectId as string
    "chunk_ids": []
}

try:
    knowledge_obj = KnowledgeObject(**knowledge_object_data)
    insert_knowledge_object(knowledge_obj)
    print("✅ Knowledge object created successfully")
except Exception as e:
    print(f"❌ Knowledge object creation failed: {e}")

# 4. Process and insert Chunks with embeddings
chunk_ids = []

for i, chunk_dict in enumerate(all_chunks):
    try:
        # Skip chunks with no content
        if chunk_dict["content"] == "<Not Found>" or not chunk_dict["content"].strip():
            print(f"⏭️ Skipping empty chunk {i+1}")
            continue
            
        # Generate embedding for the chunk content
        print(f"🔄 Generating embedding for chunk {i+1}/{len(all_chunks)}...")
        embedding = get_openai_embedding(chunk_dict["content"])
        
        # Create embedding metadata
        embedding_meta = create_embedding_meta(
            model_name="text-embedding-3-small",
            dimensionality=len(embedding),
            embedding_method="azure_openai"
        )
        
        # Prepare chunk data for Pydantic model (matching the expected schema)
        chunk_data = {
            "document_id": module_id,  # Use module_id (ObjectId as string) for document_id
            "chunk_id": i,
            "chunk_start": 0,  # You can calculate actual start positions if needed
            "chunk_end": len(chunk_dict["content"]),
            "chunk_text": chunk_dict["content"],
            "embedding": embedding,
            "embedding_meta": embedding_meta
        }
        
        # Create Chunk object and insert
        chunk_obj = Chunk(**chunk_data)
        chunk_id = insert_chunk(chunk_obj)
        
        if chunk_id:
            chunk_ids.append(chunk_id)
            print(f"✅ Inserted chunk {i+1}/{len(all_chunks)}: {chunk_dict['section'][:50]}...")
        else:
            print(f"❌ Failed to insert chunk {i+1}")
            
    except Exception as e:
        print(f"❌ Error processing chunk {i+1}: {e}")
        continue

print(f"\n🎉 Successfully processed {len(chunk_ids)} out of {len(all_chunks)} chunks")

# 5. Update Knowledge Object with chunk IDs
try:
    from scripts.mongo_utils import update_knowledge_object_chunk_ids
    update_knowledge_object_chunk_ids(
        module_id=module_id,
        title=knowledge_object_data["title"],
        chunk_ids=chunk_ids
    )
    print("✅ Updated knowledge object with chunk IDs")
except Exception as e:
    print(f"❌ Failed to update knowledge object with chunk IDs: {e}")

print(f"\n📊 Summary:")
print(f"  - Module: {module_id}")
print(f"  - Knowledge Object: {knowledge_object_data['title']}")
print(f"  - Chunks processed: {len(chunk_ids)}")
print(f"  - MongoDB collections updated: modules, knowledge_objects, chunks")

✅ Inserted module: 68cbf1e48bc9122af55fc36f
✅ Module created with ID: 68cbf1e48bc9122af55fc36f


C:\Users\jimmy\AppData\Local\Temp\ipykernel_27116\2720541164.py:37: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  modified_time=datetime.utcnow(),


✅ Upserted knowledge object for module: 68cbf1e48bc9122af55fc36f
✅ Knowledge object created successfully
🔄 Generating embedding for chunk 1/29...
✅ Inserted chunk: 68cbf1e68bc9122af55fc370
✅ Inserted chunk 1/29: Agreement Metadata...
🔄 Generating embedding for chunk 2/29...
✅ Inserted chunk: 68cbf1e68bc9122af55fc371
✅ Inserted chunk 2/29: Scope of Work...
🔄 Generating embedding for chunk 3/29...
✅ Inserted chunk: 68cbf1e78bc9122af55fc372
✅ Inserted chunk 3/29: Staffing/Personnel Requirements...
🔄 Generating embedding for chunk 4/29...
✅ Inserted chunk: 68cbf1e78bc9122af55fc373
✅ Inserted chunk 4/29: Staffing/Personnel Requirements...
🔄 Generating embedding for chunk 5/29...
✅ Inserted chunk: 68cbf1e88bc9122af55fc374
✅ Inserted chunk 5/29: Staffing/Personnel Requirements...
🔄 Generating embedding for chunk 6/29...
✅ Inserted chunk: 68cbf1e88bc9122af55fc375
✅ Inserted chunk 6/29: Staffing/Personnel Requirements...
🔄 Generating embedding for chunk 7/29...
✅ Inserted chunk: 68cbf1e88bc9122

In [1]:
import pdfplumber
from pathlib import Path

def simple_pdf_display_plumber(pdf_path, max_pages=5):
    """Simple PDF reader using pdfplumber to display content up to specified pages"""
    
    # Check if file exists
    if not Path(pdf_path).exists():
        print(f"❌ PDF file not found: {pdf_path}")
        return
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            
            print("=" * 60)
            print(f"📄 PDF CONTENT DISPLAY (PDFPlumber)")
            print(f"📁 File: {Path(pdf_path).name}")
            print(f"📊 Total pages: {total_pages}")
            print(f"👁️  Showing first {min(max_pages, total_pages)} pages")
            print("=" * 60)
            
            # Display pages
            for page_num in range(min(max_pages, total_pages)):
                page = pdf.pages[page_num]
                page_text = page.extract_text()
                
                if page_text:  # Only display if text is found
                    print(f"\n{'🔹' * 20} PAGE {page_num + 1} {'🔹' * 20}")
                    print(f"Character count: {len(page_text)}")
                    print(f"Word count: {len(page_text.split())}")
                    print("-" * 50)
                    print(page_text)
                    print("-" * 50)
                    
                    # Check for tables on this page
                    tables = page.extract_tables()
                    if tables:
                        print(f"📊 Found {len(tables)} table(s) on this page")
                        for i, table in enumerate(tables[:2]):  # Show first 2 tables
                            print(f"\nTable {i+1}:")
                            for row in table[:3]:  # Show first 3 rows
                                print(f"  {row}")
                            if len(table) > 3:
                                print(f"  ... and {len(table) - 3} more rows")
                else:
                    print(f"\n{'🔹' * 20} PAGE {page_num + 1} {'🔹' * 20}")
                    print("⚠️ No text found on this page (might be an image or scanned page)")
                
                if page_num < min(max_pages, total_pages) - 1:
                    print(f"\n{'⬇️' * 15} NEXT PAGE {'⬇️' * 15}\n")
            
            # Summary
            print(f"\n{'✅' * 20} SUMMARY {'✅' * 20}")
            print(f"Successfully displayed {min(max_pages, total_pages)} out of {total_pages} pages")
            
    except Exception as e:
        print(f"❌ Error reading PDF: {e}")

# Usage example
if __name__ == "__main__":
    # Replace with your PDF path
    pdf_path = r"C:\Users\jimmy\Downloads\Neerukonda_Reservoir.pdf"
    
    # Display first 5 pages
    simple_pdf_display_plumber(pdf_path, max_pages=20)

📄 PDF CONTENT DISPLAY (PDFPlumber)
📁 File: Neerukonda_Reservoir.pdf
📊 Total pages: 246
👁️  Showing first 20 pages

🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹 PAGE 1 🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
Character count: 420
Word count: 59
--------------------------------------------------
Amaravati Development Corporation Limited (ADCL)
NIT NO. 303/ADCL/IPD/2024-25 DATED: 31.12.2024
Name of the Work: WORLD BANK & ADB financed Amaravati Capital City
Development Program related ‘Balance flood mitigation
works of Construction of Neerukonda Reservoir (0.4 TMC) in
Amaravati Capital City, Andhra Pradesh on Lump sum
contract (Percentage Tender) including DLP of 2 Years
(Package III)’.
ISSUED ON: 31.12.2024
1
--------------------------------------------------

⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️ NEXT PAGE ⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️


🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹 PAGE 2 🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
Character count: 1696
Word count: 113
--------------------------------------------------
Standard Bidding Document
Table of Contents
NOTICE INVITING TEND

In [ ]:
# Quick usage examples
from pathlib import Path
import pyPDF2

# 1. Simple text extraction
def simple_pdf_read(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# 2. Page-by-page extraction
def extract_by_pages(pdf_path):
    pages_data = []
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for i, page in enumerate(reader.pages):
            pages_data.append({
                'page_number': i + 1,
                'text': page.extract_text(),
                'word_count': len(page.extract_text().split())
            })
    return pages_data

# 3. Extract with metadata
def extract_with_metadata(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        metadata = reader.metadata
        
        return {
            'title': metadata.get('/Title', 'Unknown'),
            'author': metadata.get('/Author', 'Unknown'),
            'pages': len(reader.pages),
            'text': ''.join([page.extract_text() for page in reader.pages])
        }

In [6]:
import PyPDF2
import pdfplumber
import re
from transformers import GPT2TokenizerFast
from datetime import datetime
import json

# Load tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def extract_tender_document(pdf_path):
    """
    Extract and chunk tender/bidding documents with specialized sections
    """
    
    # Extract text using pdfplumber for better table handling
    full_text = ""
    tables_data = []
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages):
                page_text = page.extract_text()
                if page_text:
                    full_text += f"\n--- Page {page_num + 1} ---\n"
                    full_text += page_text + "\n"
                
                # Extract tables
                page_tables = page.extract_tables()
                if page_tables:
                    for table_num, table in enumerate(page_tables):
                        tables_data.append({
                            'page': page_num + 1,
                            'table_num': table_num + 1,
                            'data': table,
                            'context': page_text[:200] if page_text else ""
                        })
    except Exception as e:
        print(f"Error with pdfplumber, falling back to PyPDF2: {e}")
        # Fallback to PyPDF2
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num, page in enumerate(reader.pages):
                page_text = page.extract_text()
                full_text += f"\n--- Page {page_num + 1} ---\n"
                full_text += page_text + "\n"
    
    return full_text, tables_data

def extract_tender_metadata(text):
    """Extract key tender information"""
    metadata = {}
    
    # NIT Number and Date
    nit_match = re.search(r'NIT\s+NO[\.:]?\s*(\d+/[A-Z]+/[A-Z]+/\d{4}-\d{2})', text, re.IGNORECASE)
    if nit_match:
        metadata['nit_number'] = nit_match.group(1)
    
    date_match = re.search(r'DATED:\s*(\d{2}\.\d{2}\.\d{4})', text, re.IGNORECASE)
    if date_match:
        metadata['nit_date'] = date_match.group(1)
    
    # Project Name
    project_match = re.search(r'Name of.*?Work[:\s]*(.+?)(?=\n|\r|ISSUED)', text, re.DOTALL | re.IGNORECASE)
    if project_match:
        metadata['project_name'] = project_match.group(1).strip()
    
    # Contract Value
    value_match = re.search(r'(?:Estimated\s+Contract\s+Value|ECV)[:\s]*Rs\.?\s*([\d,\.]+)\s*Crores?', text, re.IGNORECASE)
    if value_match:
        metadata['contract_value'] = value_match.group(1)
    
    # Completion Period
    period_match = re.search(r'Period of Completion.*?(\d+)\s*Months?', text, re.IGNORECASE)
    if period_match:
        metadata['completion_period'] = period_match.group(1) + " Months"
    
    # Organization
    org_match = re.search(r'(AMARAVATI DEVELOPMENT CORPORATION LIMITED|ADCL)', text, re.IGNORECASE)
    if org_match:
        metadata['organization'] = org_match.group(1)
    
    return metadata

def define_tender_sections():
    """Define section patterns for tender documents"""
    return {
        "Notice_Inviting_Tender": {
            "patterns": [
                r"Notice Inviting Tender.*?(?=PART\s+1|Section\s+I|\n\n\d+)",
                r"NIT.*?(?=PART\s+1|Section\s+I)"
            ],
            "priority": "critical",
            "chunk_size": 1200,
            "extract_tables": True
        },
        "Instructions_to_Bidders": {
            "patterns": [
                r"Section\s+I\s*-\s*Instructions to Bidders.*?(?=Section\s+II|SECTION\s+II)",
                r"Instructions to Bidders.*?(?=Section\s+II|Bid Data Sheet)"
            ],
            "priority": "high",
            "chunk_size": 1000,
            "subsections": [
                "Scope of Bid", "Sources of Funds", "Fraud and Corruption", 
                "Eligible Bidders", "Bid Security", "Period of Validity"
            ]
        },
        "Bid_Data_Sheet": {
            "patterns": [
                r"Section\s+II\s*-\s*Bid Data Sheet.*?(?=Section\s+III|SECTION\s+III)",
                r"BID DATA SHEET.*?(?=Section\s+III|EVALUATION)"
            ],
            "priority": "high",
            "chunk_size": 800,
            "extract_tables": True
        },
        "Evaluation_Criteria": {
            "patterns": [
                r"Section\s+III\s*-\s*Evaluation.*?(?=Section\s+IV|SECTION\s+IV)",
                r"EVALUATION AND QUALIFICATION CRITERIA.*?(?=Section\s+IV|BIDDING FORMS)"
            ],
            "priority": "high",
            "chunk_size": 1000
        },
        "Bidding_Forms": {
            "patterns": [
                r"Section\s+IV\s*-\s*Bidding Forms.*?(?=Section\s+V|SECTION\s+V)",
                r"BIDDING FORMS.*?(?=Section\s+V|FRAUD)"
            ],
            "priority": "medium",
            "chunk_size": 800
        },
        "Works_Requirements": {
            "patterns": [
                r"Section\s+VI\s*-\s*Works.*?Requirements.*?(?=PART\s+3|Section\s+VII)",
                r"WORKS.*REQUIREMENTS.*?(?=PART\s+3|GENERAL CONDITIONS)"
            ],
            "priority": "high",
            "chunk_size": 1200
        },
        "Contract_Conditions": {
            "patterns": [
                r"Section\s+VII\s*-\s*General Conditions.*?(?=Section\s+VIII|SECTION\s+VIII)",
                r"GENERAL CONDITIONS OF CONTRACT.*?(?=Section\s+VIII|PARTICULAR CONDITIONS)"
            ],
            "priority": "medium",
            "chunk_size": 1000
        }
    }

def extract_subsections(text, section_name, subsection_list):
    """Extract specific subsections within a section"""
    subsections = {}
    
    for subsection in subsection_list:
        # Create flexible pattern for subsection
        pattern = rf"(?:\d+\.?\s*)?{re.escape(subsection)}.*?(?=(?:\d+\.?\s*)?(?:{'|'.join(subsection_list)})|$)"
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        if match:
            subsections[subsection] = match.group().strip()
    
    return subsections

def chunk_tender_section(text, section_config, section_name):
    """Chunk a tender section based on its configuration"""
    chunks = []
    chunk_size = section_config.get("chunk_size", 1000)
    priority = section_config.get("priority", "medium")
    
    # Handle subsections if specified
    if "subsections" in section_config:
        subsections = extract_subsections(text, section_name, section_config["subsections"])
        
        for subsection_name, subsection_text in subsections.items():
            if len(subsection_text.strip()) > 50:  # Only process substantial content
                sub_chunks = create_semantic_chunks(
                    subsection_text, 
                    f"{section_name}_{subsection_name}", 
                    chunk_size
                )
                for chunk in sub_chunks:
                    chunk.update({
                        "priority": priority,
                        "parent_section": section_name,
                        "subsection": subsection_name
                    })
                chunks.extend(sub_chunks)
    else:
        # Standard chunking
        chunks = create_semantic_chunks(text, section_name, chunk_size)
        for chunk in chunks:
            chunk.update({
                "priority": priority,
                "parent_section": section_name
            })
    
    return chunks

def create_semantic_chunks(text, section_name, max_chunk_size=1000):
    """Create semantically meaningful chunks"""
    # First try to split by numbered sections (1., 2., etc.)
    numbered_sections = re.split(r'\n(?=\d+\.)', text)
    
    chunks = []
    current_chunk = ""
    
    for section in numbered_sections:
        section = section.strip()
        if not section:
            continue
        
        # Check if adding this section would exceed chunk size
        tokens = tokenizer.encode(current_chunk + "\n" + section)
        
        if len(tokens) > max_chunk_size and current_chunk:
            # Save current chunk and start new one
            chunks.append({
                "section": section_name,
                "content": current_chunk.strip(),
                "token_count": len(tokenizer.encode(current_chunk)),
                "metadata": {"chunk_type": "semantic"}
            })
            current_chunk = section
        else:
            current_chunk += "\n" + section if current_chunk else section
    
    # Add final chunk
    if current_chunk.strip():
        chunks.append({
            "section": section_name,
            "content": current_chunk.strip(),
            "token_count": len(tokenizer.encode(current_chunk)),
            "metadata": {"chunk_type": "semantic"}
        })
    
    return chunks

def process_tender_tables(tables_data):
    """Process extracted tables into searchable chunks"""
    table_chunks = []
    
    for table_info in tables_data:
        if not table_info['data']:
            continue
        
        # Convert table to text
        table_text = f"Table from Page {table_info['page']}:\n"
        table_text += f"Context: {table_info['context']}\n\n"
        
        for row in table_info['data']:
            if row and any(cell for cell in row if cell and cell.strip()):
                # Clean and join row data
                clean_row = [str(cell).strip() if cell else "" for cell in row]
                table_text += " | ".join(clean_row) + "\n"
        
        table_chunks.append({
            "section": f"Table_Page_{table_info['page']}",
            "content": table_text,
            "metadata": {
                "chunk_type": "table",
                "page": table_info['page'],
                "table_number": table_info['table_num']
            },
            "priority": "medium"
        })
    
    return table_chunks

def main_tender_processing(pdf_path):
    """Main function to process tender document"""
    print("🔄 Extracting tender document...")
    
    # Extract text and tables
    full_text, tables_data = extract_tender_document(pdf_path)
    
    # Extract metadata
    doc_metadata = extract_tender_metadata(full_text)
    print(f"📊 Document Metadata: {doc_metadata}")
    
    # Get section definitions
    sections_config = define_tender_sections()
    
    all_chunks = []
    
    # Process each section
    for section_name, config in sections_config.items():
        print(f"🔍 Processing section: {section_name}")
        
        # Try each pattern to find the section
        section_text = None
        for pattern in config["patterns"]:
            match = re.search(pattern, full_text, re.DOTALL | re.IGNORECASE)
            if match:
                section_text = match.group().strip()
                break
        
        if section_text and len(section_text.strip()) > 100:
            # Chunk the section
            section_chunks = chunk_tender_section(section_text, config, section_name)
            all_chunks.extend(section_chunks)
            print(f"✅ {section_name}: {len(section_chunks)} chunks created")
        else:
            # Create placeholder for missing section
            all_chunks.append({
                "section": section_name,
                "content": "<Section not found or too short>",
                "metadata": {"chunk_type": "placeholder"},
                "priority": config.get("priority", "low")
            })
            print(f"⚠️ {section_name}: Section not found")
    
    # Process tables
    print("🔄 Processing tables...")
    table_chunks = process_tender_tables(tables_data)
    all_chunks.extend(table_chunks)
    
    # Add document metadata to all chunks
    for chunk in all_chunks:
        chunk["document_metadata"] = doc_metadata
        chunk["document_type"] = "tender_document"
    
    return all_chunks, doc_metadata

# Usage
if __name__ == "__main__":
    pdf_path = r"C:\Users\jimmy\Downloads\Neerukonda_Reservoir.pdf"
    
    # Process the tender document
    chunks, metadata = main_tender_processing(pdf_path)
    
    # Save results
    output_data = {
        "document_metadata": metadata,
        "total_chunks": len(chunks),
        "chunks": chunks
    }
    
    output_path = pdf_path.replace('.pdf', '_tender_chunks.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n📊 Processing Summary:")
    print(f"  - Total chunks created: {len(chunks)}")
    print(f"  - Document type: Tender/Bidding Document")
    print(f"  - Project: {metadata.get('project_name', 'N/A')}")
    print(f"  - Contract Value: {metadata.get('contract_value', 'N/A')} Crores")
    print(f"  - Results saved to: {output_path}")
    
    # Show chunk distribution
    section_counts = {}
    for chunk in chunks:
        section = chunk.get("section", "Unknown")
        section_counts[section] = section_counts.get(section, 0) + 1
    
    print(f"\n📈 Chunk Distribution:")
    for section, count in section_counts.items():
        print(f"  - {section}: {count} chunks")

c:\Users\jimmy\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


🔄 Extracting tender document...
📊 Document Metadata: {'nit_number': '303/ADCL/IPD/2024-25', 'nit_date': '31.12.2024', 'project_name': 'WORLD BANK & ADB financed Amaravati Capital City', 'contract_value': '470.75', 'completion_period': '24 Months', 'organization': 'Amaravati Development Corporation Limited'}
🔍 Processing section: Notice_Inviting_Tender
✅ Notice_Inviting_Tender: 1 chunks created
🔍 Processing section: Instructions_to_Bidders
✅ Instructions_to_Bidders: 0 chunks created
🔍 Processing section: Bid_Data_Sheet
✅ Bid_Data_Sheet: 1 chunks created
🔍 Processing section: Evaluation_Criteria
✅ Evaluation_Criteria: 1 chunks created
🔍 Processing section: Bidding_Forms
✅ Bidding_Forms: 1 chunks created
🔍 Processing section: Works_Requirements
✅ Works_Requirements: 1 chunks created
🔍 Processing section: Contract_Conditions
✅ Contract_Conditions: 1 chunks created
🔄 Processing tables...

📊 Processing Summary:
  - Total chunks created: 194
  - Document type: Tender/Bidding Document
  - Proj

# Section Based Chunking

In [2]:
import re
import json
from typing import Dict, List, Tuple, Optional
from transformers import GPT2TokenizerFast
import pdfplumber
import PyPDF2
from pathlib import Path

# Load tokenizer for token counting
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

class SectionBasedChunker:
    """
    Advanced section-based chunking for various document types
    """
    
    def __init__(self, default_chunk_size: int = 1000, overlap: int = 100):
        self.default_chunk_size = default_chunk_size
        self.overlap = overlap
        self.tokenizer = tokenizer
    
    def extract_pdf_content(self, pdf_path: str) -> Tuple[str, List[Dict]]:
        """Extract text and tables from PDF using pdfplumber with PyPDF2 fallback"""
        full_text = ""
        tables_data = []
        
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page_num, page in enumerate(pdf.pages):
                    page_text = page.extract_text()
                    if page_text:
                        full_text += f"\n--- Page {page_num + 1} ---\n"
                        full_text += page_text + "\n"
                    
                    # Extract tables
                    page_tables = page.extract_tables()
                    if page_tables:
                        for table_num, table in enumerate(page_tables):
                            tables_data.append({
                                'page': page_num + 1,
                                'table_num': table_num + 1,
                                'data': table,
                                'context': page_text[:200] if page_text else ""
                            })
        except Exception as e:
            print(f"Error with pdfplumber, falling back to PyPDF2: {e}")
            # Fallback to PyPDF2
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page_num, page in enumerate(reader.pages):
                    page_text = page.extract_text()
                    full_text += f"\n--- Page {page_num + 1} ---\n"
                    full_text += page_text + "\n"
        
        return full_text, tables_data
    
    def get_section_patterns(self, document_type: str = "general") -> Dict:
        """Define section patterns for different document types"""
        
        patterns = {
            "tender": {
                "Notice_Inviting_Tender": {
                    "patterns": [
                        r"Notice\s+Inviting\s+Tender.*?(?=PART\s+[1I]|Section\s+[1I]|\n\n\d+\.)",
                        r"NIT\s+NO.*?(?=PART\s+[1I]|Section\s+[1I])",
                        r"(?:^|\n).*?NOTICE.*?TENDER.*?(?=Section|PART|\n\d+\.)"
                    ],
                    "priority": "critical",
                    "chunk_size": 1200
                },
                "Instructions_to_Bidders": {
                    "patterns": [
                        r"Section\s+[1I]\s*[-:]?\s*Instructions\s+to\s+Bidders.*?(?=Section\s+[2II])",
                        r"Instructions\s+to\s+Bidders.*?(?=Section\s+[2II]|Bid\s+Data\s+Sheet)"
                    ],
                    "priority": "high",
                    "chunk_size": 1000,
                    "subsections": ["Scope of Bid", "Eligible Bidders", "Bid Security", "Period of Validity"]
                },
                "Bid_Data_Sheet": {
                    "patterns": [
                        r"Section\s+[2II]\s*[-:]?\s*Bid\s+Data\s+Sheet.*?(?=Section\s+[3III])",
                        r"BID\s+DATA\s+SHEET.*?(?=Section\s+[3III]|EVALUATION)"
                    ],
                    "priority": "high",
                    "chunk_size": 800
                },
                "Technical_Specifications": {
                    "patterns": [
                        r"(?:Section\s+)?(?:[5V]|VI)\s*[-:]?\s*(?:Technical\s+)?Specifications.*?(?=Section|ANNEXURE|\n\d+\.)",
                        r"TECHNICAL\s+SPECIFICATIONS.*?(?=Section|ANNEXURE)"
                    ],
                    "priority": "high",
                    "chunk_size": 1200
                },
                "Schedule_of_Rates": {
                    "patterns": [
                        r"Schedule\s+of\s+Rates.*?(?=Section|ANNEXURE|\n\d+\.)",
                        r"SCHEDULE.*?RATES.*?(?=Section|ANNEXURE)",
                        r"Bill\s+of\s+Quantities.*?(?=Section|ANNEXURE)"
                    ],
                    "priority": "high",
                    "chunk_size": 800,
                    "extract_tables": True
                }
            },
            
            "agreement": {
                "Agreement_Metadata": {
                    "patterns": [
                        r"(?:^|\n).*?Agreement.*?between.*?(?=SCOPE|Article|Section|\n\d+\.)",
                        r"THIS\s+AGREEMENT.*?(?=SCOPE|NOW\s+THEREFORE)"
                    ],
                    "priority": "critical",
                    "chunk_size": 1000
                },
                "Scope_of_Work": {
                    "patterns": [
                        r"(?:SCOPE|Article\s+1).*?(?:of\s+)?Work.*?(?=Article|Section|\n\d+\.)",
                        r"SCOPE\s+OF\s+SERVICES.*?(?=Article|Section)"
                    ],
                    "priority": "high",
                    "chunk_size": 1200
                },
                "Terms_and_Conditions": {
                    "patterns": [
                        r"(?:Article|Section).*?Terms.*?Conditions.*?(?=Article|Section|\n\d+\.)",
                        r"TERMS\s+AND\s+CONDITIONS.*?(?=Article|Section)"
                    ],
                    "priority": "high",
                    "chunk_size": 1000
                },
                "Payment_Terms": {
                    "patterns": [
                        r"(?:Article|Section).*?Payment.*?(?=Article|Section|\n\d+\.)",
                        r"PAYMENT\s+TERMS.*?(?=Article|Section)"
                    ],
                    "priority": "high",
                    "chunk_size": 800
                }
            },
            
            "technical": {
                "Introduction": {
                    "patterns": [
                        r"(?:^|\n)(?:1\.?\s*)?Introduction.*?(?=2\.|Section|Chapter|\n\d+\.)",
                        r"INTRODUCTION.*?(?=SCOPE|TECHNICAL|Section)"
                    ],
                    "priority": "medium",
                    "chunk_size": 1000
                },
                "Technical_Requirements": {
                    "patterns": [
                        r"(?:Section|Chapter).*?Technical\s+Requirements.*?(?=Section|Chapter|\n\d+\.)",
                        r"TECHNICAL\s+REQUIREMENTS.*?(?=Section|Chapter)"
                    ],
                    "priority": "high",
                    "chunk_size": 1200
                },
                "Implementation": {
                    "patterns": [
                        r"(?:Section|Chapter).*?Implementation.*?(?=Section|Chapter|\n\d+\.)",
                        r"IMPLEMENTATION.*?(?=Section|Chapter)"
                    ],
                    "priority": "high",
                    "chunk_size": 1000
                }
            },
            
            "general": {
                "Executive_Summary": {
                    "patterns": [
                        r"(?:Executive\s+)?Summary.*?(?=Introduction|Section|Chapter|\n\d+\.)",
                        r"EXECUTIVE\s+SUMMARY.*?(?=INTRODUCTION|Section)"
                    ],
                    "priority": "high",
                    "chunk_size": 1000
                },
                "Introduction": {
                    "patterns": [
                        r"(?:1\.?\s*)?Introduction.*?(?=2\.|Section|Chapter|\n\d+\.)",
                        r"INTRODUCTION.*?(?=Section|Chapter)"
                    ],
                    "priority": "medium",
                    "chunk_size": 1000
                },
                "Methodology": {
                    "patterns": [
                        r"(?:Section|Chapter).*?Methodology.*?(?=Section|Chapter|\n\d+\.)",
                        r"METHODOLOGY.*?(?=Section|Chapter)"
                    ],
                    "priority": "high",
                    "chunk_size": 1200
                },
                "Conclusion": {
                    "patterns": [
                        r"(?:Section|Chapter).*?Conclusion.*?(?=Section|Chapter|References|\n\d+\.)",
                        r"CONCLUSION.*?(?=References|ANNEXURE)"
                    ],
                    "priority": "medium",
                    "chunk_size": 1000
                }
            }
        }
        
        return patterns.get(document_type, patterns["general"])
    
    def detect_document_type(self, text: str) -> str:
        """Auto-detect document type based on content"""
        text_lower = text.lower()
        
        # Check for tender documents
        tender_keywords = ["notice inviting tender", "nit", "bid data sheet", "evaluation criteria", "bidding forms"]
        if any(keyword in text_lower for keyword in tender_keywords):
            return "tender"
        
        # Check for agreement documents
        agreement_keywords = ["agreement", "this agreement", "scope of work", "terms and conditions"]
        if any(keyword in text_lower for keyword in agreement_keywords):
            return "agreement"
        
        # Check for technical documents
        technical_keywords = ["technical requirements", "implementation", "specifications", "system design"]
        if any(keyword in text_lower for keyword in technical_keywords):
            return "technical"
        
        return "general"
    
    def extract_section_content(self, text: str, patterns: List[str]) -> Optional[str]:
        """Extract section content using multiple patterns"""
        for pattern in patterns:
            match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
            if match:
                return match.group().strip()
        return None
    
    def extract_subsections(self, text: str, subsection_list: List[str]) -> Dict[str, str]:
        """Extract specific subsections within a section"""
        subsections = {}
        
        for subsection in subsection_list:
            # Create flexible pattern for subsection
            pattern = rf"(?:\d+\.?\s*)?{re.escape(subsection)}.*?(?=(?:\d+\.?\s*)?(?:{'|'.join(subsection_list)})|$)"
            match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
            if match:
                subsections[subsection] = match.group().strip()
        
        return subsections
    
    def create_semantic_chunks(self, text: str, section_name: str, chunk_size: int = None) -> List[Dict]:
        """Create semantically meaningful chunks with token counting"""
        if chunk_size is None:
            chunk_size = self.default_chunk_size
        
        # Try different splitting strategies
        chunks = []
        
        # Strategy 1: Split by numbered sections (1., 2., etc.)
        numbered_sections = re.split(r'\n(?=\d+\.)', text)
        
        # Strategy 2: Split by lettered sections (a), b), etc.)
        if len(numbered_sections) == 1:
            numbered_sections = re.split(r'\n(?=\([a-z]\))', text)
        
        # Strategy 3: Split by paragraphs if no clear structure
        if len(numbered_sections) == 1:
            numbered_sections = re.split(r'\n\s*\n', text)
        
        current_chunk = ""
        chunk_count = 0
        
        for section in numbered_sections:
            section = section.strip()
            if not section:
                continue
            
            # Check if adding this section would exceed chunk size
            test_text = current_chunk + "\n" + section if current_chunk else section
            tokens = self.tokenizer.encode(test_text)
            
            if len(tokens) > chunk_size and current_chunk:
                # Save current chunk and start new one
                chunk_count += 1
                chunks.append({
                    "section": section_name,
                    "chunk_id": chunk_count,
                    "content": current_chunk.strip(),
                    "token_count": len(self.tokenizer.encode(current_chunk)),
                    "word_count": len(current_chunk.split()),
                    "metadata": {
                        "chunk_type": "semantic",
                        "section_based": True
                    }
                })
                current_chunk = section
            else:
                current_chunk = test_text
        
        # Add final chunk
        if current_chunk.strip():
            chunk_count += 1
            chunks.append({
                "section": section_name,
                "chunk_id": chunk_count,
                "content": current_chunk.strip(),
                "token_count": len(self.tokenizer.encode(current_chunk)),
                "word_count": len(current_chunk.split()),
                "metadata": {
                    "chunk_type": "semantic",
                    "section_based": True
                }
            })
        
        return chunks
    
    def process_tables(self, tables_data: List[Dict]) -> List[Dict]:
        """Process extracted tables into searchable chunks"""
        table_chunks = []
        
        for table_info in tables_data:
            if not table_info['data']:
                continue
            
            # Convert table to text
            table_text = f"Table from Page {table_info['page']}:\n"
            table_text += f"Context: {table_info['context']}\n\n"
            
            for row_idx, row in enumerate(table_info['data']):
                if row and any(cell for cell in row if cell and str(cell).strip()):
                    # Clean and join row data
                    clean_row = [str(cell).strip() if cell else "" for cell in row]
                    table_text += " | ".join(clean_row) + "\n"
            
            table_chunks.append({
                "section": f"Table_Page_{table_info['page']}_{table_info['table_num']}",
                "chunk_id": 1,
                "content": table_text,
                "token_count": len(self.tokenizer.encode(table_text)),
                "word_count": len(table_text.split()),
                "metadata": {
                    "chunk_type": "table",
                    "page": table_info['page'],
                    "table_number": table_info['table_num'],
                    "section_based": True
                },
                "priority": "medium"
            })
        
        return table_chunks
    
    def chunk_document(self, pdf_path: str, document_type: str = None) -> Tuple[List[Dict], Dict]:
        """Main function to chunk document by sections"""
        print(f"🔄 Processing document: {Path(pdf_path).name}")
        
        # Extract content
        full_text, tables_data = self.extract_pdf_content(pdf_path)
        
        # Auto-detect document type if not provided
        if document_type is None:
            document_type = self.detect_document_type(full_text)
        
        print(f"📄 Document type detected: {document_type}")
        
        # Get section patterns
        section_patterns = self.get_section_patterns(document_type)
        
        all_chunks = []
        sections_found = {}
        
        # Process each section
        for section_name, config in section_patterns.items():
            print(f"🔍 Processing section: {section_name}")
            
            # Extract section content
            section_text = self.extract_section_content(full_text, config["patterns"])
            
            if section_text and len(section_text.strip()) > 50:
                # Handle subsections if specified
                if "subsections" in config:
                    subsections = self.extract_subsections(section_text, config["subsections"])
                    
                    if subsections:
                        for subsection_name, subsection_text in subsections.items():
                            if len(subsection_text.strip()) > 50:
                                sub_chunks = self.create_semantic_chunks(
                                    subsection_text, 
                                    f"{section_name}_{subsection_name}",
                                    config.get("chunk_size", self.default_chunk_size)
                                )
                                for chunk in sub_chunks:
                                    chunk.update({
                                        "priority": config.get("priority", "medium"),
                                        "parent_section": section_name,
                                        "subsection": subsection_name,
                                        "document_type": document_type
                                    })
                                all_chunks.extend(sub_chunks)
                        sections_found[section_name] = len(subsections)
                    else:
                        # No subsections found, chunk normally
                        section_chunks = self.create_semantic_chunks(
                            section_text, 
                            section_name,
                            config.get("chunk_size", self.default_chunk_size)
                        )
                        for chunk in section_chunks:
                            chunk.update({
                                "priority": config.get("priority", "medium"),
                                "document_type": document_type
                            })
                        all_chunks.extend(section_chunks)
                        sections_found[section_name] = len(section_chunks)
                else:
                    # Standard section chunking
                    section_chunks = self.create_semantic_chunks(
                        section_text, 
                        section_name,
                        config.get("chunk_size", self.default_chunk_size)
                    )
                    for chunk in section_chunks:
                        chunk.update({
                            "priority": config.get("priority", "medium"),
                            "document_type": document_type
                        })
                    all_chunks.extend(section_chunks)
                    sections_found[section_name] = len(section_chunks)
                
                print(f"✅ {section_name}: {sections_found[section_name]} chunks created")
            else:
                # Create placeholder for missing section
                all_chunks.append({
                    "section": section_name,
                    "chunk_id": 1,
                    "content": f"<{section_name} section not found or content too short>",
                    "token_count": 0,
                    "word_count": 0,
                    "metadata": {
                        "chunk_type": "placeholder",
                        "section_based": True
                    },
                    "priority": "low",
                    "document_type": document_type
                })
                print(f"⚠️ {section_name}: Section not found")
        
        # Process tables
        if tables_data:
            print(f"🔄 Processing {len(tables_data)} table(s)...")
            table_chunks = self.process_tables(tables_data)
            all_chunks.extend(table_chunks)
        
        # Generate summary
        summary = {
            "document_type": document_type,
            "total_chunks": len(all_chunks),
            "sections_found": sections_found,
            "table_chunks": len(tables_data),
            "total_tokens": sum(chunk.get("token_count", 0) for chunk in all_chunks),
            "processing_timestamp": str(datetime.now())
        }
        
        return all_chunks, summary

# Usage Examples
def example_usage():
    """Examples of how to use the SectionBasedChunker"""
    
    # Initialize chunker
    chunker = SectionBasedChunker(default_chunk_size=1000, overlap=100)
    
    # Example 1: Auto-detect document type and chunk
    pdf_path = r"C:\Users\jimmy\Downloads\Neerukonda_Reservoir_1.pdf"
    chunks, summary = chunker.chunk_document(pdf_path)
    
    print(f"📊 Processing Summary:")
    print(f"  - Document type: {summary['document_type']}")
    print(f"  - Total chunks: {summary['total_chunks']}")
    print(f"  - Total tokens: {summary['total_tokens']}")
    
    # Example 2: Specify document type
    chunks_tender, summary_tender = chunker.chunk_document(pdf_path, document_type="tender")
    
    # Example 3: Save results
    output_data = {
        "summary": summary,
        "chunks": chunks
    }
    
    output_path = pdf_path.replace('.pdf', '_section_chunks.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)
    
    print(f"💾 Results saved to: {output_path}")
    
    # Show chunk distribution by section
    section_counts = {}
    priority_counts = {"critical": 0, "high": 0, "medium": 0, "low": 0}
    
    for chunk in chunks:
        section = chunk.get("section", "Unknown")
        priority = chunk.get("priority", "medium")
        
        section_counts[section] = section_counts.get(section, 0) + 1
        priority_counts[priority] += 1
    
    print(f"\n📈 Chunk Distribution by Section:")
    for section, count in sorted(section_counts.items()):
        print(f"  - {section}: {count} chunks")
    
    print(f"\n🎯 Chunk Distribution by Priority:")
    for priority, count in priority_counts.items():
        print(f"  - {priority}: {count} chunks")
    
    return chunks, summary

# Custom section patterns for specific use cases
def add_custom_patterns():
    """Example of how to add custom section patterns"""
    
    custom_patterns = {
        "project_report": {
            "Project_Overview": {
                "patterns": [
                    r"Project\s+Overview.*?(?=Background|Objectives|Section|\n\d+\.)",
                    r"OVERVIEW.*?(?=BACKGROUND|OBJECTIVES)"
                ],
                "priority": "high",
                "chunk_size": 1200
            },
            "Financial_Analysis": {
                "patterns": [
                    r"Financial\s+Analysis.*?(?=Risk|Conclusion|Section|\n\d+\.)",
                    r"FINANCIAL.*?ANALYSIS.*?(?=RISK|CONCLUSION)"
                ],
                "priority": "critical",
                "chunk_size": 800,
                "extract_tables": True
            },
            "Risk_Assessment": {
                "patterns": [
                    r"Risk\s+Assessment.*?(?=Mitigation|Conclusion|Section|\n\d+\.)",
                    r"RISK.*?ASSESSMENT.*?(?=MITIGATION|CONCLUSION)"
                ],
                "priority": "high",
                "chunk_size": 1000
            }
        }
    }
    
    return custom_patterns

if __name__ == "__main__":
    from datetime import datetime
    
    # Run example
    chunks, summary = example_usage()

🔄 Processing document: Neerukonda_Reservoir_1.pdf
📄 Document type detected: tender
🔍 Processing section: Notice_Inviting_Tender
✅ Notice_Inviting_Tender: 1 chunks created
🔍 Processing section: Instructions_to_Bidders
✅ Instructions_to_Bidders: 1 chunks created
🔍 Processing section: Bid_Data_Sheet
✅ Bid_Data_Sheet: 1 chunks created
🔍 Processing section: Technical_Specifications
✅ Technical_Specifications: 1 chunks created
🔍 Processing section: Schedule_of_Rates
✅ Schedule_of_Rates: 5 chunks created
🔄 Processing 43 table(s)...
📊 Processing Summary:
  - Document type: tender
  - Total chunks: 52
  - Total tokens: 28471
🔄 Processing document: Neerukonda_Reservoir_1.pdf
📄 Document type detected: tender
🔍 Processing section: Notice_Inviting_Tender
✅ Notice_Inviting_Tender: 1 chunks created
🔍 Processing section: Instructions_to_Bidders
✅ Instructions_to_Bidders: 1 chunks created
🔍 Processing section: Bid_Data_Sheet
✅ Bid_Data_Sheet: 1 chunks created
🔍 Processing section: Technical_Specificati